In [2]:
spark

In [3]:
# Indexing string columns for ALS
indexer = StringIndexer(inputCol="user_id", outputCol="user_idx")
scores_df = indexer.fit(scores_df).transform(scores_df)

indexer = StringIndexer(inputCol="anime_id", outputCol="anime_idx")
scores_df = indexer.fit(scores_df).transform(scores_df)


In [4]:
# ALS model parameters
als = ALS(maxIter=5, regParam=0.01, userCol="user_idx", itemCol="anime_idx", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

# Train the ALS model
model = als.fit(scores_df)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(scores_df)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


24/05/03 02:14:22 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:14:22 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:14:33 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:14:46 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:14:51 WARN BlockManager: Block rdd_64_4 could not be removed as it was not found on disk or in memory
24/05/03 02:14:51 WARN BlockManager: Block rdd_64_8 could not be removed as it was not found on disk or in memory
24/05/03 02:14:51 WARN BlockManager: Block rdd_64_3 could not be removed as it was not found on disk or in memory
24/05/03 02:14:52 WARN BlockManager: Block rdd_65_3 could not be removed as it was not found on disk or in memory
24/05/03 02:14:52 WARN BlockManager: Block rdd_65_4 could not be removed as it was not found on disk or in memory
24/05/03 02:14:52 WARN BlockManager: Block rdd_65_8 could not be removed as it was not

24/05/03 02:14:57 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 7.0 in stage 15.0 (TID 89),5,main]
java.lang.OutOfMemoryError: Java heap space
24/05/03 02:14:57 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 0.0 in stage 15.0 (TID 82),5,main]
java.lang.OutOfMemoryError: Java heap space
24/05/03 02:14:57 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 9.0 in stage 15.0 (TID 91),5,main]
java.lang.OutOfMemoryError: Java heap space
24/05/03 02:14:57 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 2.0 in stage 15.0 (TID 84),5,main]
java.lang.OutOfMemoryError: Java heap space
24/05/03 02:14:57 ERROR SparkUncaughtExceptionHandler: Uncaught exception in thread Thread[Executor task launch worker for task 6.0 in stage 15.0 (TID 88),5,main]
java.lang

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 54] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while send

ConnectionRefusedError: [Errno 61] Connection refused

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

# Initialize Spark session with increased memory settings
spark = SparkSession.builder \
    .appName("Anime Recommendation System") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Data paths
anime_data_path = "/Users/kaushik/Desktop/Data/anime-dataset-2023_new.csv"
user_details_path = "/Users/kaushik/Desktop/Data/users-details-2023_new.csv"
user_scores_path = "/Users/kaushik/Desktop/Data/users-score-2023_new.csv"

# Read the datasets
anime_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("escape", '"').load(anime_data_path)
users_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("escape", '"').load(user_details_path)
scores_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("escape", '"').load(user_scores_path)

# Indexing string columns for ALS
indexer = StringIndexer(inputCol="user_id", outputCol="user_idx")
scores_df = indexer.fit(scores_df).transform(scores_df)

indexer = StringIndexer(inputCol="anime_id", outputCol="anime_idx")
scores_df = indexer.fit(scores_df).transform(scores_df)

# Repartition scores_df to manage memory better and avoid data skew
scores_df = scores_df.repartition(200)

# Cache the DataFrame to optimize read performance
scores_df.persist()

# ALS model parameters
als = ALS(maxIter=5, regParam=0.01, userCol="user_idx", itemCol="anime_idx", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True, blockSize=4096)

# Train the ALS model
model = als.fit(scores_df)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(scores_df)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 anime recommendations for each user
user_recs = model.recommendForAllUsers(10)
user_recs.show()

# Clean up cached data
scores_df.unpersist()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 02:22:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/03 02:22:52 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:22:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , user_id, Username, anime_id, Anime Title, rating
 Schema: _c0, user_id, Username, anime_id, Anime Title, rating
Expected: _c0 but found: 
CSV file: file:///Users/kaushik/Desktop/Data/users-score-2023_new.csv
24/05/03 02:23:09 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
24/05/03 02:23:22 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/05/03 02:23:23 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/05/03 02:23:28 WARN DAGScheduler: Broadcasting large task binary with size 2.

Root-mean-square error = 1.5131798000292254


24/05/03 02:24:34 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
24/05/03 02:24:58 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


+--------+--------------------+
|user_idx|     recommendations|
+--------+--------------------+
|      28|[{16495, 35.53374...|
|      31|[{16495, 43.22482...|
|      34|[{16495, 44.04334...|
|      53|[{16495, 65.13637...|
|      65|[{16495, 54.89232...|
|      78|[{16495, 35.98358...|
|      81|[{16495, 42.61424...|
|      85|[{16495, 42.57969...|
|     101|[{16495, 44.97214...|
|     108|[{16495, 45.18070...|
|     115|[{16495, 39.83249...|
|     126|[{16495, 39.19075...|
|     133|[{16495, 44.8857}...|
|     137|[{16495, 48.79183...|
|     148|[{16495, 52.17040...|
|     155|[{16495, 40.66792...|
|     183|[{16495, 44.80581...|
|     193|[{16495, 42.88467...|
|     210|[{16495, 45.71884...|
|     211|[{16495, 50.24815...|
+--------+--------------------+
only showing top 20 rows



DataFrame[_c0: int, user_id: int, Username: string, anime_id: int, Anime Title: string, rating: int, user_idx: double, anime_idx: double]

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import explode, col

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Anime Recommendation System") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Data paths
anime_data_path = "/Users/kaushik/Desktop/Data/anime-dataset-2023_new.csv"
user_details_path = "/Users/kaushik/Desktop/Data/users-details-2023_new.csv"
user_scores_path = "/Users/kaushik/Desktop/Data/users-score-2023_new.csv"

# Read the datasets
anime_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("escape", '"').load(anime_data_path)
users_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("escape", '"').load(user_details_path)
scores_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("escape", '"').load(user_scores_path)

# Indexing the anime and user IDs in scores_df
anime_indexer = StringIndexer(inputCol="anime_id", outputCol="anime_idx", handleInvalid="skip")
scores_df = anime_indexer.fit(scores_df).transform(scores_df)

user_indexer = StringIndexer(inputCol="user_id", outputCol="user_idx", handleInvalid="skip")
scores_df = user_indexer.fit(scores_df).transform(scores_df)

# Indexing the anime IDs in anime_df for joining later
anime_df = anime_indexer.fit(anime_df).transform(anime_df)

# ALS model parameters
als = ALS(maxIter=5, regParam=0.01, userCol="user_idx", itemCol="anime_idx", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

# Train the ALS model
model = als.fit(scores_df)

# Generate recommendations
user_recs = model.recommendForAllUsers(10)

# Explode the recommendations into separate rows
expanded_recs = user_recs.withColumn("recommendations", explode("recommendations")).select(
    col("user_idx"),
    col("recommendations.anime_idx").alias("anime_idx"),
    col("recommendations.rating").alias("predicted_rating")
)

# Join with the anime DataFrame to get the titles
final_recommendations = expanded_recs.join(anime_df, "anime_idx").select(
    "user_idx", "Name", "predicted_rating"
)

# Show the final recommendations
final_recommendations.show(truncate=False)

# Evaluate the model by computing the RMSE
predictions = model.transform(scores_df)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Stop the Spark session
spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 02:53:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/03 02:54:05 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:05 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:14 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:18 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:22 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:23 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:25 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:54:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/03 02:54:26 

+--------+------------------------------------------------------------+----------------+
|user_idx|Name                                                        |predicted_rating|
+--------+------------------------------------------------------------+----------------+
|28      |Haha Xiong Juchang: Zhihui Da Jiangtang                     |52.258476       |
|28      |Amrita no Kyouen                                            |41.644794       |
|28      |Heya Camp△: Sauna to Gohan to Sanrin Bike                   |37.71732        |
|28      |Buggy Map                                                   |34.61948        |
|28      |Choujigen Game Neptune: Hi☆Light                            |30.94444        |
|28      |Yuu☆Yuu☆Hakusho: Eizou Hakusho - Opening Ending Encyclopedia|27.65014        |
|28      |Ling Jian Zun 3rd Season                                    |27.508638       |
|28      |Zen-chan Two-chan                                           |27.48463        |
|28      |Oshiri Tant

24/05/03 02:55:10 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:55:11 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:55:11 WARN DAGScheduler: Broadcasting large task binary with size 8.4 MiB
24/05/03 02:55:20 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/05/03 02:55:27 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB
24/05/03 02:55:33 WARN DAGScheduler: Broadcasting large task binary with size 8.5 MiB


Root-mean-square error = 1.7251361645877417
